In [1]:
import pandas as pd
import json
import requests

## Migration data from IRS

In [2]:
migration_df = pd.read_csv('Data/IRS_county_to_county_migration_2017-2021.csv')
migration_df.head()

,year,dest_state_id,dest_county_id,orig_state_id,orig_county_id,dest_state,dest_county,returns,individuals,adj_gross_income
0,2017,1,1,96,0,AL,Autauga County Total Migration-US and Foreign,2400,5702,125069000
1,2017,1,1,97,0,AL,Autauga County Total Migration-US,2366,5573,122696000
2,2017,1,1,97,1,AL,Autauga County Total Migration-Same State,1516,3352,66187000
3,2017,1,1,97,3,AL,Autauga County Total Migration-Different State,850,2221,56510000
4,2017,1,1,98,0,AL,Autauga County Total Migration-Foreign,34,129,2373000


### Restructure the data to include new single-column County ID's 

In [3]:
# create a new index column 'dest_county_id' for each county
migration_df['new_county_id'] = pd.factorize(migration_df['dest_state_id'].astype(str) + '_' + migration_df['dest_county_id'].astype(str))[0]

In [4]:
# create a new sorting column 'inflow_type'

def get_inflow_type(row):
    if row['orig_state_id'] == 96 and row['orig_county_id'] == 0:
        return 'us_and_foreign'
    elif row['orig_state_id'] == 97 and row['orig_county_id'] == 0:
        return 'us'
    elif row['orig_state_id'] == 97 and row['orig_county_id'] == 1:
        return 'same_state'
    elif row['orig_state_id'] == 97 and row['orig_county_id'] == 3:
        return 'different_state'
    elif row['orig_state_id'] == 98 and row['orig_county_id'] == 0:
        return 'foreign'
    elif row['dest_county_id'] == row['orig_county_id']:
        return 'non-migrants'
    elif row['orig_state_id'] == 57 and row['orig_county_id'] == 9:
        return 'other_foreign'
    elif row['orig_state_id'] == 58 and row['orig_county_id'] == 0:
        return 'other_same_state'
    elif row['orig_state_id'] == 59 and row['orig_county_id'] == 0:
        return 'other_diff_state'
    else:
        return 'us_counties'  # Default value for unmatched cases

# Add the 'inflow_type' column based on the custom function
migration_df['inflow_type'] = migration_df.apply(get_inflow_type, axis=1)


In [5]:
migration_df.head()

,year,dest_state_id,dest_county_id,orig_state_id,orig_county_id,dest_state,dest_county,returns,individuals,adj_gross_income,new_county_id,inflow_type
0,2017,1,1,96,0,AL,Autauga County Total Migration-US and Foreign,2400,5702,125069000,0,us_and_foreign
1,2017,1,1,97,0,AL,Autauga County Total Migration-US,2366,5573,122696000,0,us
2,2017,1,1,97,1,AL,Autauga County Total Migration-Same State,1516,3352,66187000,0,same_state
3,2017,1,1,97,3,AL,Autauga County Total Migration-Different State,850,2221,56510000,0,different_state
4,2017,1,1,98,0,AL,Autauga County Total Migration-Foreign,34,129,2373000,0,foreign


## Create new Counties table / df

In [6]:
import re

# Create an empty DataFrame to store the results
counties_df = pd.DataFrame(columns=['county_id', 'county', 'state', 'fips_state', 'fips_county'])

# Iterate over the unique new_county_id in migration_df
for dest_county_id in migration_df['new_county_id'].unique():
    # Select the first row with the current dest_county_id
    row = migration_df[migration_df['new_county_id'] == dest_county_id].iloc[0]
    # Extract the state and county information from the selected row
    state = row['dest_state']
    county = re.split(' County| Parish| City| Borough| Municipality', row['dest_county'], flags=re.IGNORECASE)[0]
    fips_st = row['dest_state_id']
    fips_ct = row['dest_county_id']
    # Create a temporary DataFrame and concatenate it with counties_df
    temp_df = pd.DataFrame({
        'county_id': [dest_county_id],
        'county': [county],
        'state': [state],
        'fips_state': [fips_st],
        'fips_county': [fips_ct]
    })
    counties_df = pd.concat([counties_df, temp_df], ignore_index=True).reset_index(drop=True)

counties_df = counties_df[['county_id', 'county', 'state', 'fips_state', 'fips_county']]

In [7]:
# remove all rows showing county name as 'census'
counties_df = counties_df[~counties_df['county'].str.contains('census', case=False, na=False)]

# remove all rows with no county identified (from 2017, 2018 data)
counties_df = counties_df[~counties_df['county'].str.startswith('Total')]

In [8]:
# save counties_df to csv
counties_df.sort_values(by='county')
counties_df.to_csv('Data/us_counties', index=False)

In [9]:
counties_df = counties_df.reset_index(drop=True)
counties_df

,county_id,county,state,fips_state,fips_county
0,0,Autauga,AL,1,1
1,1,Baldwin,AL,1,3
2,2,Barbour,AL,1,5
3,3,Bibb,AL,1,7
4,4,Blount,AL,1,9
...,...,...,...,...,...
3126,3187,Teton,WY,56,39
3127,3188,Uinta,WY,56,41
3128,3189,Washakie,WY,56,43
3129,3190,Weston,WY,56,45


## Migration table restructuring

### Add new county id's 

In [10]:
# Perform a merge operation to add 'orig_county' name 
migration_df = migration_df.merge(
    counties_df, 
    how='left', 
    left_on=['orig_state_id', 'orig_county_id'], 
    right_on=['fips_state', 'fips_county']
)

In [11]:
migration_df = migration_df[['year', 'new_county_id', 'inflow_type', 'county_id', 'county', 'state', 'returns', 'individuals', 'adj_gross_income', ]]
migration_df = migration_df.rename(columns={'new_county_id': 'dest_county_id', 'county_id': 'orig_county_id', 'county': 'orig_county', 'state': 'orig_state'})
migration_df.head()

,year,dest_county_id,inflow_type,orig_county_id,orig_county,orig_state,returns,individuals,adj_gross_income
0,2017,0,us_and_foreign,NaN,NaN,NaN,2400,5702,125069000
1,2017,0,us,NaN,NaN,NaN,2366,5573,122696000
2,2017,0,same_state,NaN,NaN,NaN,1516,3352,66187000
3,2017,0,different_state,NaN,NaN,NaN,850,2221,56510000
4,2017,0,foreign,NaN,NaN,NaN,34,129,2373000


In [12]:
# Perform a merge operation to add 'dest_county' 
migration_df = migration_df.merge(counties_df, 
                                  left_on='dest_county_id', 
                                  right_on='county_id', 
                                  how='left')
migration_df.rename(columns={'state': 'dest_state', 'county': 'dest_county'}, inplace=True)

In [13]:
migration_df = migration_df[['year', 'dest_county_id', 'dest_state', 'dest_county', 'inflow_type', 'orig_county_id', 'orig_county', 'orig_state', 'returns', 'individuals', 'adj_gross_income', ]]
migration_df.fillna('', inplace=True)
migration_df.head()

,year,dest_county_id,dest_state,dest_county,inflow_type,orig_county_id,orig_county,orig_state,returns,individuals,adj_gross_income
0,2017,0,AL,Autauga,us_and_foreign,,,,2400,5702,125069000
1,2017,0,AL,Autauga,us,,,,2366,5573,122696000
2,2017,0,AL,Autauga,same_state,,,,1516,3352,66187000
3,2017,0,AL,Autauga,different_state,,,,850,2221,56510000
4,2017,0,AL,Autauga,foreign,,,,34,129,2373000


In [29]:
migration_df[(migration_df['dest_county_id'] == 227) & (migration_df['year'] == 2020)].head(50)

,year,dest_county_id,dest_state,dest_county,inflow_type,orig_county_id,orig_county,orig_state,returns,individuals,adj_gross_income
239604,2020,227,CA,San Francisco,us_and_foreign,,,,30040,38116,3762317000
239605,2020,227,CA,San Francisco,us,,,,30012,38072,3760697000
239606,2020,227,CA,San Francisco,same_state,,,,19004,24876,2382797000
239607,2020,227,CA,San Francisco,different_state,,,,11008,13196,1377900000
239608,2020,227,CA,San Francisco,foreign,,,,28,44,1620000
239609,2020,227,CA,San Francisco,non-migrants,227,San Francisco,CA,367147,618122,70130576000
239610,2020,227,CA,San Francisco,us_counties,230,San Mateo,CA,4144,5918,485595000
239611,2020,227,CA,San Francisco,us_counties,190,Alameda,CA,3682,4684,391280000
239612,2020,227,CA,San Francisco,us_counties,232,Santa Clara,CA,3314,3991,603016000
239613,2020,227,CA,San Francisco,us_counties,208,Los Angeles,CA,1698,2010,180594000


In [30]:
len(migration_df)

385052